# Librerias

In [12]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import plotly.graph_objs as go
from collections import defaultdict
import statistics
import json
import re
import matplotlib.pyplot as plt
from datetime import timedelta
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import xml.etree.ElementTree as ET


# Codigo de main

In [13]:
import re
import datetime, time

#Esta clase sirve para parsear timestamps, recibe 2 y hace la diferencia
class Tiempo:
    def __init__ (self, inicio, final = -1, seconds = -1): #Necesario para tener 2 constructores
        self.positivo = True
        if final == -1: #Primer constructor (1 argumento)

            self.hours = 0
            self.minutes = 0
            self.seconds = 0

            erHours = re.compile(r'\d*h')
            erMinutes = re.compile(r'\d*m')
            erSeconds = re.compile(r'\d*s')
            h = erHours.search(inicio)
            m = erMinutes.search(inicio)
            s = erSeconds.search(inicio)

            if h != None:
                self.hours = h[0][:-1]
            if m != None:
                self.minutes = m[0][:-1]
            if s != None:
                self.seconds = s[0][:-1]
                
            self.hours = int(self.hours)
            self.minutes = int(self.minutes)
            self.seconds = int(self.seconds)
                
            if self.seconds >= 60:
                self.minutes += int(self.seconds/60)
                self.seconds = self.seconds%60
            if self.minutes >= 60:
                self.hours += int(self.minutes/60)
                self.minutes = self.minutes%60
            
        elif seconds == -1: #Segundo constructor (2 aregumentos)
            self.hours = 0
            self.minutes = 0
            self.seconds = 0

            fin = time.mktime(datetime.datetime.strptime(final, "%Y-%m-%dT%H:%M:%S.%fZ").timetuple())
            ini = time.mktime(datetime.datetime.strptime(inicio, "%Y-%m-%dT%H:%M:%S.%fZ").timetuple())

            hoursRaw = int((fin - ini)/3600)
            minutesRaw = int((fin - ini)/60)
            secondsRaw = int(fin - ini)

            if hoursRaw != 0:
                self.hours = hoursRaw
                minutesRaw = int(((fin - ini)%3600)/60)
            if minutesRaw != 0:
                self.minutes = minutesRaw
                secondsRaw = int((fin - ini)%60)
            self.seconds = secondsRaw
            
        else: #Tercer constructor (3 argumentos)
            self.hours = inicio
            self.minutes = final
            self.seconds = seconds
            
        self.hours = int(self.hours)
        self.minutes = int(self.minutes)
        self.seconds = int(self.seconds)
            
    def toString(self):
        return self.__repr__()
    
    def __int__ (self):
        return self.hours*3600 + self.minutes*60 + self.seconds

    def __repr__(self):
        t=""
        if self.hours !=0:
            t += str(self.hours) + "h/"
        if self.minutes !=0:
            t += str(self.minutes) + "m/"
        if self.seconds !=0:
            t += str(self.seconds) + "s"
        else:
            t = t[:-1]
        if self.hours == 0 and self.minutes == 0 and self.seconds == 0:
            return "0s"
        return t
    
    def __add__(self, t): #Sobrecarga del operador +
        s = self.seconds + t.seconds
        m = int(self.minutes) + t.minutes
        h = self.hours + t.hours
        
        if s >= 60:
            s -= 60
            m += 1
        if m >= 60:
            m -= 60
            h += 1
            
        return Tiempo(h,m,s)
        
    
    def __sub__(self, t):
        sSelf = self.hours*3600 + self.minutes*60 + self.seconds
        sT = t.hours*3600 + t.minutes*60 + t.seconds
        
        return Tiempo(str(sSelf-sT) + "s")
    
    def __mul__(self, n): #Sobrecarga del operador *
        s = self.seconds * n
        m = self.minutes * n
        h = self.hours * n
        
        m += (h - int(h))*60
        s += (m - int(m))*60
        
        if s >= 60:
            m += int(s/60)
            s = s%60
        if m >= 60:
            h += int(m/60)
            m = m%60
        
        return Tiempo(h,m,s)
    
    def __truediv__(self, n): #Sobrecarga del operador /
        selfS = self.seconds
        selfM = self.minutes
        selfH = self.hours
        
        h = int(selfH/n)
        selfM += (selfH%n)*60
        m = int(selfM/n)
        selfS += (selfM%n)*60
        s = int(selfS/n)
        
        if s >= 60:
            m += int(s/60)
            s = s%60
        if m >= 60:
            h += int(m/60)
            m = m%60
        return Tiempo(int(h),int(m),int(s))
    
    def __abs__(self):
        return Tiempo(abs(self.hours), abs(self.minutes), abs(self.seconds))
    
    def __le__(self, t):   #Sobrecarga del operado <=  
        sSelf = self.hours*3600 + self.minutes*60 + self.seconds
        if type(t) == Tiempo:
            sT = t.hours*3600 + t.minutes*60 + t.seconds
        else:
            sT = t       
        return sSelf <= sT
    
    def __ge__(self, t):  #Sobrecarga del operado >=
        sSelf = self.hours*3600 + self.minutes*60 + self.seconds
        if type(t) == Tiempo:
            sT = t.hours*3600 + t.minutes*60 + t.seconds
        else:
            sT = t      
        return sSelf >= sT
    
    def __lt__(self, t):     #Sobrecarga del operado <
        sSelf = self.hours*3600 + self.minutes*60 + self.seconds
        if type(t) == Tiempo:
            sT = t.hours*3600 + t.minutes*60 + t.seconds
        else:
            sT = t        
        return sSelf < sT
    
    def __gt__(self, t):   #Sobrecarga del operado >
        sSelf = self.hours*3600 + self.minutes*60 + self.seconds
        if type(t) == Tiempo:
            sT = t.hours*3600 + t.minutes*60 + t.seconds
        else:
            sT = t
        return sSelf > sT
    
    def total_seconds(self):
        return self.hours * 3600 + self.minutes * 60 + self.seconds


In [14]:
import pandas as pd
import json
from collections import defaultdict
import re
from utilities import Tiempo
import statistics
import re
import plotly.express as px
import plotly.graph_objects as go
import sys
import os
import xml.etree.ElementTree as ET
import keyword

sys.stdout.flush()
print("Analizando datos...")

nombreInstituto = sys.argv[1]

if not os.path.exists("./datos/" + nombreInstituto + "/plots"):
    os.makedirs("./datos/" + nombreInstituto + "/plots")

def extraerTiemposPorNivelJugador(rawData):
    
    tiempos = defaultdict(defaultdict)
    intentosNecesarios = defaultdict(defaultdict)
    inicioYFinJuego = defaultdict()
    erroresVar = defaultdict(list)
    erroresCod = defaultdict(list)
    codJugadores = defaultdict()
    codUltNivel = defaultdict()

    error1 = "No necesario crear variable"
    error2 = "Asignacion mismo valor en variable"
    error3 = "Entero o texto utilizado 2 veces"
    errorCodSucio = "Codigo muerto"

    nivelesError1Excepcion = ['variables_2', 'variables_3', 'variables_4', 'variables_6', 'variables_8', 'variables_10', 'types_2', 'basic_operators_1', 'basic_operators_3', 'basic_operators_6']
    nivelesError3Excepcion = ['tutorials_1', 'tutorials_2', 'tutorials_3', 'tutorials_4', 'tutorials_5', 'tutorials_6', 'tutorials_7', 'tutorials_8', 'tutorials_9']

    erLevel = re.compile(r'\blevel$\b')
    erIdLevel = re.compile(r'/')
    
    erInitialized = re.compile(r'\binitialized$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erAccessed = re.compile(r'\baccessed$\b')
    erProgressed = re.compile(r'\bprogressed$\b')

    erSeriousGame = re.compile(r'\bserious-game$\b')  
    erCategoryMain = re.compile(r'\bcategories_main$\b') 

    estandarNombreVar = re.compile(r'^_*[a-zA-Z][a-zA-Z0-9_]*$')
    
    fechaSesion = {"inicio" : None, "fin" : None}
    fechaSesion["inicio"] = rawData[0]["timestamp"]
    fechaSesion["fin"] = rawData[-1]["timestamp"]
    
    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        name = evento["actor"]["name"]
        timestamp = evento["timestamp"]
        objectId = evento["object"]["id"]
       
        if erLevel.search(obj): #Si el objeto de la acción es un nivel
            levelCode = erIdLevel.split(objectId)[-1]
            if levelCode != "editor_level":
                if erInitialized.search(verb): #Si la acción es inicio o reinicio
                    if "result" in evento: #Significa que ha iniciado el nivel desde el menu
                        if levelCode in tiempos[name]:
                            intentosNecesarios[name][levelCode].append({"intentos" : 1, "success" : False})
                            tiempos[name][levelCode].append({"ini" : timestamp, "fin" : None, "stars" : ""})
                        else:
                            intentosNecesarios[name][levelCode] = [{"intentos" : 1, "success" : False}]
                            tiempos[name][levelCode] = [{"ini" : timestamp, "fin" : None, "stars" : ""}]
                    else:
                        intentosNecesarios[name][levelCode][-1]["intentos"] += 1

                elif erProgressed.search(verb):
                    cod = ET.fromstring(evento['result']['extensions']['code'])
                    codJugadores[name] = cod
                    codUltNivel[name] = levelCode

                elif erCompleted.search(verb):
                    if evento["result"]["score"]["raw"] > 0 :
                        if levelCode in tiempos[name]:
                            intentosNecesarios[name][levelCode][-1]["success"] = True
                            tiempos[name][levelCode][-1]["fin"] = timestamp
                            tiempos[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]

                        if codUltNivel[name] == levelCode:
                            
                            try:
                                cod = codJugadores[name]
                            except:
                                print(evento)
                                print(timestamp)
                                return
                            #Analisis codigo limpio
                            antBloque = False
                            for child in cod:
                                if child.tag == "block":
                                    if antBloque:
                                        erroresCod[name].append({'error' : errorCodSucio, 'level' : levelCode})
                                        break
                                    else:
                                        antBloque = True
                                else:
                                    antBloque = False 
                            
                            #Analisis nombre variables

                            def nombreVarValido(nombresVar):
                                for nombre in nombresVar:
                                    if keyword.iskeyword(nombre):
                                        erroresCod[name].append({'error' : "Variable: '" + nombre + "' es una palabra reservada", 'level' : levelCode})
                                    elif not bool(estandarNombreVar.match(nombre)):
                                        erroresCod[name].append({'error' : "Variable: '" + nombre + "' no tiene un nombre que siga el estandar de programacion", 'level' : levelCode})

                            variablesDeclaradas = []
                            for child in cod.find("variables"):
                                variablesDeclaradas.append(child.text)
                            nombreVarValido(variablesDeclaradas)

                            #Analisis uso variables
                            vars = defaultdict()
                            values = []

                            for v in cod.find('variables'):
                                vars[v.text] = {"valores" : [], "usado" : 0}

                            for b in cod.find("block[@type='start_start']").iter('block'):
                                tarjeta = b.attrib['type']

                                if tarjeta == "text" or tarjeta == "math_number":
                                    valor = b.find('field').text
                                    if valor in values and levelCode not in nivelesError3Excepcion:
                                        erroresVar[name].append({'error' : error3, 'level' : levelCode})
                                    else:
                                        values.append(valor)

                                elif tarjeta == "variables_set":
                                    varName = b.find('field').text
                                    try:
                                        varValue = b.find('value').find('block').find('field').text
                                    except:
                                        #print(ET.tostring(cod, encoding='utf8', method='xml'))
                                        #print(ET.tostring(b, encoding='utf8', method='xml'))
                                        None                                

                                    if vars[varName]['valores'] and vars[varName]['usado'] < 2 and levelCode not in nivelesError1Excepcion:
                                        erroresVar[name].append({'error' : error1 + " " + varName, 'level' : levelCode})

                                    if varValue in vars[varName]['valores']:
                                        erroresVar[name].append({'error' : error2 + " " + varName, 'level' : levelCode})
                                    else:
                                        vars[varName]['valores'].append(varValue)

                                    vars[varName]['usado'] = 0

                                elif tarjeta == "variables_get":
                                    varName = b.find('field').text
                                    vars[varName]["usado"] += 1

                            for v in vars:
                                if vars[v]['usado'] < 2 and levelCode not in nivelesError1Excepcion:
                                    erroresVar[name].append({'error' : error1 + " " + v, 'level' : levelCode})

                    elif evento["result"]["score"]["raw"] == -1:
                        if levelCode in tiempos[name]:
                            tiempos[name][levelCode][-1]["fin"] = timestamp
                            tiempos[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]
                    else:
                        tiempos[name][levelCode][-1]["fin"] = timestamp
        
        elif erSeriousGame.search(obj) and erInitialized.search(verb):
            if name in inicioYFinJuego:
                inicioYFinJuego[name].append({"inicio" : timestamp, "fin" : None})
            else:
                inicioYFinJuego[name] = [{"inicio" : timestamp, "fin" : None}]
        
        if not(erAccessed.search(verb) and erCategoryMain.search(objectId)):
            try:
                inicioYFinJuego[name][-1]["fin"] = timestamp
            except:
                None
    
    return {"tiempos" : tiempos, "intentosNecesarios" : intentosNecesarios, "inicioYFinJuego" : inicioYFinJuego, "erroresVar" : erroresVar, "erroresCod" : erroresCod, "fechaSesion" : fechaSesion}

def tiempoPorNiveles_Jugador(data):
    tiemposJugados = defaultdict(defaultdict)
    tiemposJugadosSerializable = defaultdict(defaultdict)
    for player in data:
        for level in data[player]:
            for times in data[player][level]:
                if times["fin"] != None: #Si no se aborto el intento del nivel
                    timeDifference = Tiempo(times["ini"], times["fin"])
                    if level in tiemposJugados[player]:
                        tiemposJugados[player][level].append({"time" : timeDifference, "stars" : times["stars"]})
                        tiemposJugadosSerializable[player][level].append({"time" : timeDifference.toString(), "stars" : times["stars"]})
                    else:
                        tiemposJugados[player][level] = [{"time" : timeDifference, "stars" : times["stars"]}]
                        tiemposJugadosSerializable[player][level] = [{"time" : timeDifference.toString(), "stars" : times["stars"]}]
    return tiemposJugados, tiemposJugadosSerializable

def tiempoTotalJuego(inicioYFinJuego, ultNivelAlcanzado):
    tiempoTotal = defaultdict()
    for player in inicioYFinJuego:
        for time in inicioYFinJuego[player]:
            if time["fin"] != None:
                if player in tiempoTotal:
                    tiempoTotal[player] += Tiempo(time["inicio"], time["fin"])
                else:
                    tiempoTotal[player] = Tiempo(time["inicio"], time["fin"])

    for p in tiempoTotal:
        if p in ultNivelAlcanzado:
            tiempoTotal[p] = {"id" : p, "tiempo" : str(tiempoTotal[p]), "ultNivel" : ultNivelAlcanzado[p].replace("_", " ").capitalize(), "nombre" : p}
        else:
            tiempoTotal[p] = {"id" : p, "tiempo" : str(tiempoTotal[p]), "ultNivel" : "None", "nombre" : p}


    with open('./datos/'+ nombreInstituto +'/jugadores.json', 'w') as json_file:
        json.dump(tiempoTotal, json_file)

def getMediaTiempoPorNivel(tiempos, soloPrimerExito = True, tiemposOrdenados = False):
    medias = defaultdict(list)
    mediasEstrellas = defaultdict(list)
    tiempoCompletarNivelIndividual = defaultdict(defaultdict)
    
    for player in tiempos:
        for level in tiempos[player]:
            tAux = Tiempo("0s")
            for t in tiempos[player][level]:
                if t["stars"] != "" and int(t["stars"]) != -1:
                    if level in medias:
                        medias[level].append(int(tAux + t["time"]))
                        mediasEstrellas[level].append(int(t["stars"]))
                    else:
                        medias[level] = [int(tAux + t["time"])]
                        mediasEstrellas[level] = [int(t["stars"])]
                        
                    tiempoCompletarNivelIndividual[player][level] = tAux + t["time"]
                    if soloPrimerExito:
                        break
                elif t["stars"] == 0:
                    print("ALERTA")
                else:
                    tAux += t["time"]
    for m in medias:
        medias[m] = Tiempo(str(int(round(statistics.mean(medias[m]), 0))) + "s")
        
    for m in mediasEstrellas:
        mediasEstrellas[m] = statistics.mean(mediasEstrellas[m])
    
    listaNiveles = medias.keys()
    
    if tiemposOrdenados:
        medias = sorted(medias.items(), key=lambda x: x[1])
        mediasEstrellas = sorted(mediasEstrellas.items(), key=lambda x: x[1])
    else:
        medias = list(medias.items())
        mediasEstrellas = list(mediasEstrellas.items())
    
    return {"mediaTiempos" : medias, "mediaEstrellas" : mediasEstrellas, "listaNiveles" : listaNiveles, "tiemposIndividuales" : tiempoCompletarNivelIndividual}

def getUltimoNivelAlcanzado(tiempos):
    ultNivel = defaultdict()
    for player in tiempos:
        for level in tiempos[player]:
            ultNivel[player] = level
    return ultNivel

def getIntentosMedios_HastaCompletarNivel(intentosNecesarios, intentosOrdenados = False):
    intentosMedios = defaultdict(list)
    intentosCompletarNivelIndividual = defaultdict(defaultdict)
    
    for name in intentosNecesarios:
        for level in intentosNecesarios[name]:
            cont = 0
            for i in intentosNecesarios[name][level]:
                cont += i["intentos"]
                if i["success"] == True:
                    intentosMedios[level].append(cont)
                    intentosCompletarNivelIndividual[name][level] = cont
                    break
            
    for level in intentosMedios:
        intentosMedios[level] = round(statistics.mean(intentosMedios[level]), 2)
    
    if intentosOrdenados:
        intentosMedios = sorted(intentosMedios.items(), key=lambda x: x[1])
    else:
        intentosMedios = list(intentosMedios.items())
    return {"intentosMedios" : intentosMedios, "intentosIndividual" : intentosCompletarNivelIndividual}

def extraerArray(my_dict):
    nested_keys = []
    nested_values = []

    for k, v in my_dict.items():
        for nested_k, nested_v in v.items():
            nested_keys.append(nested_k)
            nested_values.append(nested_v)

    # create a dictionary with keys as the values of the nested_keys array
    nested_dict = defaultdict(list)
    for key, value in zip(nested_keys, nested_values):
        nested_dict[key].append(value)
    
    return nested_dict

def extraerArrayConNombres(my_dict):
    nested_keys = []
    nested_values = []

    for k, v in my_dict.items():
        for nested_k, nested_v in v.items():
            nested_keys.append(nested_k)
            nested_values.append({"name" : k, "valor" : nested_v})

    # create a dictionary with keys as the values of the nested_keys array
    nested_dict = defaultdict(defaultdict)
    for key, value in zip(nested_keys, nested_values):
        nested_dict[key][value["name"]] = value["valor"]
    
    return nested_dict

def parseTiemposDictToInteger(data_dict):
    parsed_dict = {}

    for key, value in data_dict.items():
        parsed_list = []
        for element in value:
            parsed_list.append(int(element))
        parsed_dict[key] = parsed_list

    return parsed_dict

def getCuantasPersonasHanAlcanzadoNivel(ultNivelCompletado, niveles):
    cuantosHanLlegadoAlNivel = defaultdict()
    
    ultNivelCopia = ultNivelCompletado.copy()

    #Contamos cuanta gente ha llegado hasta cada nivel
    for nivel in niveles:
        cuantosHanLlegadoAlNivel[nivel] = len(ultNivelCopia)
        
        keys_to_delete = [k for k, v in ultNivelCopia.items() if v == nivel]
        for k in keys_to_delete:
            del ultNivelCopia[k]

    return cuantosHanLlegadoAlNivel

def generateChartNivelesAlcanzados(niveles, ultNivelCompletado):
    cuantosHanLlegadoAlNivel = getCuantasPersonasHanAlcanzadoNivel(ultNivelCompletado, niveles)
    ultNivelCat = {}
    for level in cuantosHanLlegadoAlNivel:
        ultNivelCat[(" ".join(level.split("_")[:-1])).capitalize()] = cuantosHanLlegadoAlNivel[level]
    
    df = pd.DataFrame({"levels" : list(ultNivelCat.keys()), "nJugadores" : list(ultNivelCat.values())})
    fig = px.bar(df, x="levels", y='nJugadores', labels={'levels':'Categorías', 'nJugadores':'Número Jugadores'}, title = "Categorías superadas")
    fig.update_layout(plot_bgcolor='#C3CEDA')
    fig.update_traces(marker_color='#738FA7', hovertemplate='<b>Numero Jugadores: %{y}</b>')
    fig.write_json("./datos/" + nombreInstituto + "/plots/categoriasSuperadas.json")

def parseTiemposDictConNombresToInteger(tiemposDict):
    for l in tiemposDict:
        for j in tiemposDict[l]:
            tiemposDict[l][j] = int(tiemposDict[l][j])
    return tiemposDict

def getListaCategorias(listaNiveles):
    categorias = defaultdict()
    erroresVar = defaultdict()
    erroresCod = defaultdict()

    for n in listaNiveles:
        categorias[n.split("_")[0]] = (" ".join(n.split("_")[:-1])).capitalize()
    
    for j in resultados_Tiempos_Nivel_Jugador["erroresVar"]:
        for e in resultados_Tiempos_Nivel_Jugador["erroresVar"][j]:
            if e["level"] in erroresVar:
                erroresVar[e["level"]] += 1
            else:
                erroresVar[e["level"]] = 1

    for j in resultados_Tiempos_Nivel_Jugador["erroresCod"]:
        for e in resultados_Tiempos_Nivel_Jugador["erroresCod"][j]:
            if e["level"] in erroresCod:
                erroresCod[e["level"]] += 1
            else:
                erroresCod[e["level"]] = 1
    
    nPersonasNivel = getCuantasPersonasHanAlcanzadoNivel(ultNivelAlcanzado, tiemposMedios["listaNiveles"])
    listVar = []
    listCod = []
    for n in nPersonasNivel:
        if n in erroresVar:
            listVar.append({n : {"errores" : erroresVar[n], "jugadores" : nPersonasNivel[n]}})
        else:
            listVar.append({n : {"errores" : 0, "jugadores" : nPersonasNivel[n]}})

        if n in erroresCod:
            listCod.append({n : {"errores" : erroresCod[n], "jugadores" : nPersonasNivel[n]}})
        else:
            listCod.append({n : {"errores" : 0, "jugadores" : nPersonasNivel[n]}})


    listVar = sorted(listVar, key=lambda x: x[list(x.keys())[0]]['errores'], reverse=True)
    listCod = sorted(listCod, key=lambda x: x[list(x.keys())[0]]['errores'], reverse=True)

    with open("./datos/" + nombreInstituto + "/info.json", 'w') as json_file:
        json.dump({"categorias" : categorias, "niveles" : list(listaNiveles), "nErroresVar" : listVar, "nErroresCod" : listCod, "fechaSesion" : resultados_Tiempos_Nivel_Jugador["fechaSesion"], "nJugadores" : len(ultNivelAlcanzado)}, json_file)
    return categorias

def create_boxplots(data_dict, titulo):
    categorias = defaultdict(defaultdict)
    for d in data_dict:
        categorias[(" ".join(d.split("_")[:-1]))][d] = data_dict[d]

    for c in categorias:
        df = pd.DataFrame.from_dict(categorias[c], orient="index")
        df = df.reset_index()
        df = df.rename(columns={'index' : 'Niveles'})
        df_melted = df.melt(id_vars=['Niveles'], var_name='Jugador', value_name=titulo)
        # create boxplot
        fig = px.box(df_melted, x='Niveles', y=titulo, hover_name='Jugador', title = c.capitalize())
        fig.update_layout(plot_bgcolor='#C3CEDA')
        fig.update_traces(marker_color='#738FA7', hovertemplate='<b>%{hovertext}</b><br>' + titulo + ': %{y}')
        fig.write_json("./datos/" + nombreInstituto + "/plots/"+ c.split(" ")[0] + "_"+ titulo +".json")

def getChartsComparativas(niveles, tiemposMedios, ultNivelCompletado, jugClase):
    cuantosHanLlegadoAlNivel = getCuantasPersonasHanAlcanzadoNivel(ultNivelCompletado, niveles)
    ultNivelCat = {}
    for level in cuantosHanLlegadoAlNivel:
        ultNivelCat[(" ".join(level.split("_")[:-1])).capitalize()] = cuantosHanLlegadoAlNivel[level]

    for c in ultNivelCat:
        ultNivelCat[c] = round((ultNivelCat[c]/jugClase)*100, 2)

    categorias = defaultdict(defaultdict)
    for d in tiemposMedios:
        categorias[(" ".join(d[0].split("_")[:-1])).capitalize()][d[0]] = int(d[1])


def getMediaCategorias(df_tiempo, df_intentos, df_estrellas):
    data = defaultdict(defaultdict)
    tiempo = defaultdict()
    intentos = defaultdict()
    estrellas = defaultdict()

    nPersonasNivel = getCuantasPersonasHanAlcanzadoNivel(ultNivelAlcanzado, tiemposMedios["listaNiveles"])

    for l in df_tiempo:
        tiempo[l[0]] = l[1].toString()

    for l in df_intentos:
        intentos[l[0]] = round(l[1], 2)
    
    for l in df_estrellas:
        estrellas[l[0]] = round(l[1],2)

    for l in tiempo:
        c = l.split("_")[0]
        data[c][l] = {"tiempo" : tiempo[l], "intentos" : intentos[l], "estrellas" : estrellas[l], "participantes" : nPersonasNivel[l]}

    with open("./datos/" + nombreInstituto + "/datosMedios.json", 'w') as json_file:
        json.dump(data, json_file)

    return data

def getDatosMediosPorCategoria(df_tiempo, df_intentos):
    data = defaultdict()
    tiempo = defaultdict()
    intentos = defaultdict()

    for l in df_tiempo:
        tiempo[l[0]] = l[1]

    for l in df_intentos:
        intentos[l[0]] = round(l[1], 2)
    
    for l in tiempo:
        c = (" ".join(l.split("_")[:-1])).capitalize()
        if c in data:
            data[c] = {"tiempo" : data[c]["tiempo"] + tiempo[l], "intentos" : round(data[c]["intentos"] + intentos[l], 2)}
        else:
            data[c] = {"tiempo" : tiempo[l], "intentos" : intentos[l]}
    
    for c in data:
        data[c]["tiempo"] = data[c]["tiempo"].toString()


    return data

#####################################################################################################################

JSONFile = open('../Web/datos/1/trazasOrdenadas.json')
rawData = json.load(JSONFile)
JSONFile.close()

resultados_Tiempos_Nivel_Jugador = extraerTiemposPorNivelJugador(rawData)

with open('./datos/'+ nombreInstituto +'/erroresVar.json', 'w') as json_file:
    json.dump(resultados_Tiempos_Nivel_Jugador["erroresVar"], json_file)

with open('./datos/'+ nombreInstituto +'/erroresCod.json', "w") as f:
    json.dump(resultados_Tiempos_Nivel_Jugador["erroresCod"], f)

tiemposIntentosJugadores, tiemposIntentosJugadoresSerializable = tiempoPorNiveles_Jugador(resultados_Tiempos_Nivel_Jugador["tiempos"])
with open('./datos/'+ nombreInstituto +'/tiemposIntentosJugador.json', "w") as f:
    json.dump({"tiempo" : tiemposIntentosJugadoresSerializable, "intentos" : resultados_Tiempos_Nivel_Jugador["intentosNecesarios"]}, f)

soloPrimerExito = True
tiemposOrdenados = False
tiemposMedios = getMediaTiempoPorNivel(tiemposIntentosJugadores, soloPrimerExito, tiemposOrdenados)

ultNivelAlcanzado = getUltimoNivelAlcanzado(tiemposMedios["tiemposIndividuales"])
tiempoTotalJuego(resultados_Tiempos_Nivel_Jugador["inicioYFinJuego"], ultNivelAlcanzado)

categorias = getListaCategorias(tiemposMedios["listaNiveles"])

intentosOrdenados = False
intentosMedios_Individual = getIntentosMedios_HastaCompletarNivel(resultados_Tiempos_Nivel_Jugador["intentosNecesarios"], intentosOrdenados)

tiemposList = extraerArray(tiemposMedios["tiemposIndividuales"])
intentosList = extraerArray(intentosMedios_Individual["intentosIndividual"])

intentosListNombres = extraerArrayConNombres(intentosMedios_Individual["intentosIndividual"])
tiemposListNombres = extraerArrayConNombres(tiemposMedios["tiemposIndividuales"])

generateChartNivelesAlcanzados(tiemposMedios["listaNiveles"], ultNivelAlcanzado)

create_boxplots(intentosListNombres, 'Intentos')

create_boxplots(parseTiemposDictConNombresToInteger(tiemposListNombres), "Tiempo(s)")

getChartsComparativas(tiemposMedios["listaNiveles"], tiemposMedios["mediaTiempos"], ultNivelAlcanzado, len(ultNivelAlcanzado))

getMediaCategorias(tiemposMedios["mediaTiempos"], intentosMedios_Individual["intentosMedios"], tiemposMedios["mediaEstrellas"])

getDatosMediosPorCategoria(tiemposMedios["mediaTiempos"], intentosMedios_Individual["intentosMedios"])

print("Datos analizados con exito")


Analizando datos...
Datos analizados con exito


In [15]:
#Si tenerEnCuentaMove = False -> cuando se realiza una accion de tipo "move" no se tiene en cuenta para calcular el tiempo entre interacciones, como si no existiese la traza
def extraerTiemposEntreInteracciones2(rawData, tenerEnCuentaMove = False):
    ultimaInteraccionJugador = defaultdict()
    
    tiempoInicio_Interaccion_Jugador = defaultdict(defaultdict) #Tiempo entre inicio de nivel y la siguiente interaccion (reinicio, completado o interaccion con tarjeta)
    tiempoInteraccion_Completado_Jugador = defaultdict(defaultdict) #Tiempo entre una interaccion (inicio, reinicio o interaccion con tarjeta) y terminar el nivel
    tiempoInteraccion_Interaccion_Jugador = defaultdict(defaultdict) #Tiempo entre una interaccion con tarjeta y otra interaccion con tarjeta
    
    anomalias = []
    
    erInteracted = re.compile(r'\binteracted$\b')
    erGameObject = re.compile(r'\bgame-object$\b')
    erLevelExitButton = re.compile(r'\blevel_exit_button$\b') 
    erInitialized = re.compile(r'\binitialized$\b')
    erSeriousGame = re.compile(r'\bserious-game$\b')    
    erCompleted = re.compile(r'\bcompleted$\b')
    erLevel = re.compile(r'\blevel$\b')

        
    for e in rawData:
        verb = e["verb"]["id"]
        obj = e["object"]["definition"]["type"]
        objectId = e["object"]["id"]
        name = e["actor"]["name"]
        timestamp = e["timestamp"]
        
        erIdLevel = re.compile(r'/')
        levelCode = erIdLevel.split(objectId)[-1]
        
        if(levelCode != "editor_level"):
            
            if erSeriousGame.search(obj) and erInitialized.search(verb): #Si inicia el juego se reinicia su ultima interaccion
                if name in ultimaInteraccionJugador:
                    del ultimaInteraccionJugador[name]

            elif erLevel.search(obj):
                if erInitialized.search(verb): #Significa que ha iniciado o reiniciado el nivel
                    if "result" in e: #Inicia el nivel desde el menu
                        ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "inicio"}
                    else: #Reinicia el nivel
                        try:
                            t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)
                            
                            if ultimaInteraccionJugador[name]["accion"] == "inicio":
                                #Añadimos la dif de tiempo al diccionario del jugador
                                if levelCode in tiempoInicio_Interaccion_Jugador[name]:
                                    tiempoInicio_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion" : "reinicio"})
                                else:
                                    tiempoInicio_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion" : "reinicio"}]
                            #Actualizamos ultima interaccion
                            ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "reinicio"}
                        except:
                            #Si entra aqui es porque ha reiniciado un nivel que no habia iniciado
                            #Trazas estan mal
                            anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo" : "Se ha reiniciado un nivel que no estaba iniciado"})

                elif erCompleted.search(verb) and e["result"]["score"]["raw"] != 0:
                    #Si completa un nivel o sale al menu se reinicia su ultima interaccion, si falla el intento pero vuelve a intentarlo no se reinicia
                    #Si raw == 0 significa que se reinicia el nivel
                    try:
                        t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)                        
                        #Añadimos la dif de tiempo al diccionario del jugador
                        if ultimaInteraccionJugador[name]["accion"] == "inicio":
                                #Añadimos la dif de tiempo al diccionario del jugador
                            if levelCode in tiempoInicio_Interaccion_Jugador[name]:
                                tiempoInicio_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion" : "terminado", "exito" : e["result"]["success"]})
                            else:
                                tiempoInicio_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion" : "terminado", "exito" : e["result"]["success"]}]
                                
                            if levelCode in tiempoInteraccion_Completado_Jugador[name]:
                                tiempoInteraccion_Completado_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "inicio"})
                            else:
                                tiempoInteraccion_Completado_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "inicio"}]
                                    
                        elif ultimaInteraccionJugador[name]["accion"] == "reinicio":
                            if levelCode in tiempoInteraccion_Completado_Jugador[name]:
                                tiempoInteraccion_Completado_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "reinicio"})
                            else:
                                tiempoInteraccion_Completado_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "reinicio"}]

                        elif ultimaInteraccionJugador[name]["accion"] == "interaccion":
                            if levelCode in tiempoInteraccion_Completado_Jugador[name]:
                                tiempoInteraccion_Completado_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "interaccionTarjeta"})
                            else:
                                tiempoInteraccion_Completado_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "interaccionTarjeta"}]
                        #Actualizamos ultima interaccion, la borramos
                        del ultimaInteraccionJugador[name]
                    except:
                        #Si entra aqui es porque se ha completado un nivel que no se habia iniciado
                        #Probablemente por anomalias en las trazas
                        anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo": "Se ha completado un nivel que no estaba iniciado"})

            elif erGameObject.search(obj) and erInteracted.search(verb):
                if "result" in e and "extensions" in e["result"] and "level" in e["result"]["extensions"]:
                    levelCode = e["result"]["extensions"]["level"]
                    if not erLevelExitButton.search(objectId) and levelCode != "editor_level":
                        if name in ultimaInteraccionJugador:
                            if e["result"]["extensions"]["action"] != "move" or tenerEnCuentaMove: #<-- Que accion realiza
                                t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)
                                #Añadimos la dif de tiempo al diccionario del jugador

                                if ultimaInteraccionJugador[name]["accion"] == "inicio":
                                    if levelCode in tiempoInicio_Interaccion_Jugador[name]:
                                        tiempoInicio_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion" : "interaccionTarjeta"})
                                    else:
                                        tiempoInicio_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion" : "interaccionTarjeta"}]

                                if ultimaInteraccionJugador[name]["accion"] == "interaccionTarjeta":
                                    if levelCode in tiempoInteraccion_Interaccion_Jugador[name]:
                                        tiempoInteraccion_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "interaccionTarjeta"})
                                    else:
                                        tiempoInteraccion_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "interaccionTarjeta"}]

                                ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "interaccionTarjeta"}
                        else:
                            anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo" : "Se ha interactuado en un nivel que no estaba iniciado"})
                        
                    
    # Dataframe 1: tiempoInteraccion_Completado_Jugador
    df_tiempoInteraccion_Completado_Jugador = pd.DataFrame(tiempoInteraccion_Completado_Jugador)
    df_tiempoInteraccion_Completado_Jugador = df_tiempoInteraccion_Completado_Jugador.applymap(lambda x: np.nan if not isinstance(x, list) or len(x) == 0 else int(x[-1]["tiempo"]))
    df_tiempoInteraccion_Completado_Jugador.index.name = "nivel"

    # Dataframe 2: tiempoInicio_Interaccion_Jugador
    df_tiempoInicio_Interaccion_Jugador = pd.DataFrame(tiempoInicio_Interaccion_Jugador)
    df_tiempoInicio_Interaccion_Jugador = df_tiempoInicio_Interaccion_Jugador.applymap(lambda x: np.nan if not isinstance(x, list) or len(x) == 0 else int(x[0]["tiempo"]))
    df_tiempoInicio_Interaccion_Jugador.index.name = "nivel"

    # Dataframe 3: tiempoInteraccion_Interaccion_Jugador
    df_tiempoInteraccion_Interaccion_Jugador = pd.DataFrame(tiempoInteraccion_Interaccion_Jugador)
    df_tiempoInteraccion_Interaccion_Jugador = df_tiempoInteraccion_Interaccion_Jugador.applymap(lambda x: np.nan if not isinstance(x, list) or len(x) == 0 else sum(int(tiempo["tiempo"]) for tiempo in x) / len(x))
    df_tiempoInteraccion_Interaccion_Jugador.index.name = "nivel"


    return df_tiempoInteraccion_Completado_Jugador, df_tiempoInicio_Interaccion_Jugador, df_tiempoInteraccion_Interaccion_Jugador

df_tiempoInteraccion_Completado_Jugador, df_tiempoInicio_Interaccion_Jugador, df_tiempoInteraccion_Interaccion_Jugador = extraerTiemposEntreInteracciones2(rawData, True)
#df_tiempoInteraccion_Completado_Jugador
#df_tiempoInicio_Interaccion_Jugador
#df_tiempoInteraccion_Interaccion_Jugador


In [16]:
# Crear un DataFrame con los datos de los intentos medios de cada jugador

intentosJugadores = pd.DataFrame(intentosMedios_Individual["intentosIndividual"])
#intentosJugadores

# Convertir columnas en filas
df_stacked = intentosJugadores.stack().reset_index()
df_stacked.columns = ['nivel', 'token', 'intentos']

# Calcular promedio de intentos por jugador
promedio_intentos = df_stacked.groupby('token')['intentos'].mean()

# Crear DataFrame final
df_final = pd.DataFrame({'token': promedio_intentos.index, 'intentos': promedio_intentos.values})
df_final

# Convertir índice de df_tiempoInteraccion_Interaccion_Jugador en columna
df_tiempoInteraccion_Interaccion_Jugador2 = df_tiempoInteraccion_Interaccion_Jugador.stack().reset_index()
df_tiempoInteraccion_Interaccion_Jugador2.columns = ['nivel', 'token', 'tiempo']

# Calcular promedio de intentos por jugador
promedio_interaccion = df_tiempoInteraccion_Interaccion_Jugador2.groupby('token')['tiempo'].mean()

# Crear DataFrame final
df_final2 = pd.DataFrame({'token': promedio_interaccion.index, 'tiempo': promedio_interaccion.values})

# Unir con intentosJugadores usando merge en'token'
df_jugadores = pd.merge(df_final, df_final2, on=['token'])


#Añadimos los errores:
resultados_Tiempos_Nivel_Jugador = extraerTiemposPorNivelJugador(rawData)
res = defaultdict(int)

for j in resultados_Tiempos_Nivel_Jugador["erroresCod"]:
    res[j] += len(resultados_Tiempos_Nivel_Jugador["erroresCod"][j])
for j in resultados_Tiempos_Nivel_Jugador["erroresVar"]:
    res[j] += len(resultados_Tiempos_Nivel_Jugador["erroresVar"][j])

for i, row in df_jugadores.iterrows():
    jugador = row['token']
    if jugador in res:
        df_jugadores.at[i, 'errores'] = res[jugador]
    else:
        df_jugadores.at[i, 'errores'] = 0

niveles = list(tiemposMedios["listaNiveles"])
nivel_numeros = {nivel: i+1 for i, nivel in enumerate(niveles)}

ultNivelAlcanzado = getUltimoNivelAlcanzado(tiemposMedios["tiemposIndividuales"])

for jugador, nivel in ultNivelAlcanzado.items():
    ultNivelAlcanzado[jugador] = nivel_numeros[nivel]

for i, row in df_jugadores.iterrows():
    jugador = row['token']
    if jugador in ultNivelAlcanzado:
        df_jugadores.at[i, 'nivel'] = ultNivelAlcanzado[jugador]
    else:
        df_jugadores.at[i, 'nivel'] = 0


df_jugadores


,token,intentos,tiempo,errores,nivel
0,accul,3.650000,7.387432,10.0,20.0
1,ahxym,2.875000,11.556849,5.0,16.0
2,aotlm,2.250000,8.364646,4.0,20.0
3,asopk,1.885714,5.666881,18.0,35.0
4,atlzs,2.423077,5.272170,7.0,26.0
5,auvqa,2.043478,6.953159,11.0,23.0
6,bajbm,2.277778,10.741531,7.0,18.0
7,bdwyj,2.950000,6.006920,12.0,20.0
8,bhyhw,1.892857,5.918490,17.0,28.0
9,bjvnt,2.700000,3.000000,0.0,10.0


In [17]:
# Calcular los límites de los valores atípicos
# regla empírica, que establece que los valores atípicos se encuentran a más de 3 desviaciones estándar de la media.

Q1_intentos = df_jugadores['intentos'].quantile(0.25)
Q3_intentos = df_jugadores['intentos'].quantile(0.75)
IQR_intentos = Q3_intentos - Q1_intentos
outlier_threshold_intentos = Q3_intentos + 1.5*IQR_intentos

Q1_tiempo = df_jugadores['tiempo'].quantile(0.25)
Q3_tiempo = df_jugadores['tiempo'].quantile(0.75)
IQR_tiempo = Q3_tiempo - Q1_tiempo
outlier_threshold_tiempo = Q3_tiempo + 1.5*IQR_tiempo

Q1_errores = df_jugadores['errores'].quantile(0.25)
Q3_errores = df_jugadores['errores'].quantile(0.75)
IQR_errores = Q3_errores - Q1_errores
outlier_threshold_errores = Q3_errores + 1.5*IQR_errores

df = df_jugadores[(df_jugadores['errores'] <= outlier_threshold_errores) & (df_jugadores['intentos'] <= outlier_threshold_intentos) & (df_jugadores['tiempo'] <= outlier_threshold_tiempo)]

df


,token,intentos,tiempo,errores,nivel
0,accul,3.650000,7.387432,10.0,20.0
2,aotlm,2.250000,8.364646,4.0,20.0
3,asopk,1.885714,5.666881,18.0,35.0
4,atlzs,2.423077,5.272170,7.0,26.0
5,auvqa,2.043478,6.953159,11.0,23.0
6,bajbm,2.277778,10.741531,7.0,18.0
7,bdwyj,2.950000,6.006920,12.0,20.0
8,bhyhw,1.892857,5.918490,17.0,28.0
9,bjvnt,2.700000,3.000000,0.0,10.0
10,blvkx,3.000000,10.174073,7.0,18.0


In [18]:
#PCA:

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

df = df.reset_index(drop=True)

# Seleccionamos las columnas que se van a usar en el análisis
columnas = ['intentos', 'tiempo', 'errores', 'nivel']

# Seleccionamos los datos que se van a usar en el análisis
data = df[columnas]

# Estandarizamos los datos
data_std = StandardScaler().fit_transform(data)

# Creamos un objeto PCA con el número de componentes que queremos obtener
pca = PCA(n_components=2)

# Aplicamos PCA a los datos estandarizados
componentes_principales = pca.fit_transform(data_std)

# Creamos un nuevo DataFrame con las componentes principales y los tokens
df_pca = pd.DataFrame(data = componentes_principales, columns = ['PC1', 'PC2'])
df_pca['token'] = df['token']

# Imprimimos el nuevo DataFrame
print(df_pca)


          PC1       PC2  token
0    0.615583 -0.198402  accul
1    0.955420 -1.368657  aotlm
2   -2.859834 -0.016396  asopk
3   -0.392796  0.182097  atlzs
4   -0.463336 -0.665071  auvqa
5    1.121522 -2.386778  bajbm
6   -0.040152  0.183045  bdwyj
7   -1.921290 -0.136419  bhyhw
8    2.060516  1.233477  bjvnt
9    1.308322 -1.818443  blvkx
10  -0.192002 -0.218177  buvgs
11  -1.559466 -0.432301  cmbfi
12   1.090277 -1.561771  cojqb
13  -2.189050 -0.522447  dunul
14  -1.195302 -0.737905  dwqwi
15  -0.785798  0.488738  eayrh
16   0.769935  1.511828  edxmq
17   0.266923  0.008375  ejeuw
18   3.599705  0.881769  epjcx
19   1.698375 -0.524807  epqkm
20  -1.944605  0.098550  eprdu
21  -0.274135  0.543629  ewqbs
22  -1.159444 -0.181431  fbldj
23   0.528062 -0.195014  fivin
24  -0.617914 -0.170379  fkmtp
25  -0.480891 -0.398257  fmtrq
26  -0.991567 -0.600448  fmupf
27  -2.538603  0.368433  ftpnb
28  -2.830935  0.309328  gcpze
29   1.646622  0.266173  gdljy
30   0.532832  1.779771  gguuc
31  -0.5

In [19]:
# Ruta del archivo CSV
archivo_csv = 'jugadores_pca.csv'

# Comprobar si el archivo CSV existe
if os.path.isfile(archivo_csv):
    # Cargar el archivo CSV existente
    df_pca_guardado = pd.read_csv(archivo_csv)

    # Combinar los datos guardados con los nuevos jugadores
    df_pca = pd.concat([df_pca_guardado, df_pca], ignore_index=True)
else:
    df_pca_guardado = pd.DataFrame()

# Guardar el dataframe actualizado en el archivo CSV (agregar nuevas filas)
df_pca.to_csv(archivo_csv, mode='a', header=not os.path.isfile(archivo_csv), index=False)


# con KMeans:

In [41]:
from sklearn.cluster import KMeans

#Coger las columnas necesarias para crear la matriz de características
matriz_caracteristicas = df_pca[['PC1', 'PC2']].values

# Normalizar los datos
matriz_caracteristicas_norm = StandardScaler().fit_transform(matriz_caracteristicas)

# Definimos o número de iteracións
n_iter = 20

# Obtemos os mesmos centroides iniciais que antes
SEED_VALUE = 190463  # NON CAMBIES ESTE SEED, así todos teremos os mesmos resultados
np.random.seed(SEED_VALUE)
centr_iniciais = matriz_caracteristicas_norm[np.random.choice(matriz_caracteristicas_norm.shape[0], size=5, replace=False)]

# Definimos o modelo
model = KMeans(n_clusters=len(centr_iniciais), init=centr_iniciais, n_init=1,
            max_iter=n_iter, algorithm='lloyd', random_state=SEED_VALUE)

# Axustamos o modelo aos datos
np.random.seed(SEED_VALUE)
#df
agrupamento = model.fit(matriz_caracteristicas_norm)


# Crear la figura
fig = go.Figure(data=[go.Scatter(x=df_pca['PC1'], y=df_pca['PC2'], mode='markers',
                                 marker=dict(size=10, color=agrupamento.labels_ + 1, opacity=0.8,
                                             colorscale='Viridis'),
                                              hovertext=df['token'] + '<br>' +
                                            'Tiempo: ' + df['tiempo'].astype(str) + '<br>' +
                                            'Nivel: ' + df['nivel'].astype(str) + '<br>' +
                                            'Intentos: ' + df['intentos'].astype(str) + '<br>' +
                                            'Errores: ' + df['errores'].astype(str))]) 

# Configurar layout
fig.update_layout(title='Clustering de intentos',
                  xaxis_title='PC1',
                  yaxis_title='PC2',
                  width=800, height=600)

# Mostrar gráfica
fig.show()


In [21]:
agrupamento.labels_

array([2, 3, 1, 2, 2, 3, 2, 1, 0, 3, 2, 1, 3, 1, 2, 4, 4, 2, 0, 3, 1, 4,
       2, 2, 2, 2, 2, 1, 1, 0, 4, 2, 0, 2, 0, 2, 3, 4, 3, 2, 3, 3, 2, 1,
       0, 2, 2, 0, 2, 4, 3, 1, 4, 3, 2, 2, 3, 2, 4, 2, 4, 1, 2, 0, 1, 0,
       0, 1, 4, 4, 2, 4, 1, 3, 3, 2, 4, 1, 2, 0, 4, 1, 1, 3, 2, 4, 0, 3,
       0, 0, 3, 1, 3, 2, 0, 3, 2, 2, 4, 4, 3, 0, 4, 4, 4, 4, 1, 0, 2, 4,
       1, 0, 1, 3, 3, 2, 2, 4, 3, 0, 2, 0, 2, 1, 0, 2, 2])

In [42]:
import numpy as np

# Obtener los últimos valores de los clusters en agrupamento
ultimos_clusters = agrupamento.labels_[-len(df):] +1

# Crear una nueva columna 'cluster' en el dataframe df y asignar los valores de los clusters
df['cluster'] = ultimos_clusters

# Copiar la columna 'cluster' del dataframe df a df_jugadores
df_jugadores['cluster'] = 00
for index, row in df.iterrows():
    jugador_id = row['token']  # Obtener el ID de jugador de df
    
    # Comprobar si el ID de jugador existe en df_jugadores
    if jugador_id in df_jugadores['token'].values:
        cluster_valor = row['cluster']  # Obtener el valor de la columna "cluster" de df
        
        # Asignar el valor correspondiente a la fila correspondiente en df_jugadores
        df_jugadores.loc[df_jugadores['token'] == jugador_id, 'cluster'] = cluster_valor

#print(df_jugadores)

#Cluster 0: son los outliers tanto en tiempo, como intentos, como errores. son los que necesitan mas ayuda
#Cluster 1 violeta: errores 0 o menos de 10, nivel 15 no llegan a 20, intentos 4-5-6, tiempo 4-5-6
#Cluster 2 añil: de media unos 15, nivel 25 o 30+, intentos 1-2, tiempo 4-5
#Cluster 3 medio: errores 10, nivel 20 22, tiempo 5-6 pero 1-2 intentos
#Cluster 4 verde: errores entre 3-7, nivel desde 14 a 22, tiempo 8-9-10, intentos 1-2-3
#Cluster 5 amarillo: minimo nivel 20, tiempo medio de 4-5, intentos 2-4, errores unos 10 12
jugadoresCluster = df_jugadores.to_dict()
jugadoresClusterDict = defaultdict()
for j in jugadoresCluster['token']:
    jugadoresClusterDict[jugadoresCluster['token'][j]] = jugadoresCluster['cluster'][j]
with open("./datos/prueaba.json", 'w') as json_file:
        json.dump(jugadoresClusterDict, json_file)


# con DBSCAN:

In [23]:
"""from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

#Coger las columnas necesarias para crear la matriz de características
matriz_caracteristicas = df_pca[['PC1', 'PC2']].values

# Normalizar los datos
#matriz_caracteristicas_norm = StandardScaler().fit_transform(matriz_caracteristicas)

# Definir el modelo
model = DBSCAN(eps=0.7, min_samples=5)

# Ajustar el modelo a los datos
agrupamento = model.fit(matriz_caracteristicas)

# Crear la figura
fig = go.Figure(data=[go.Scatter(x=df_pca['PC1'], y=df_pca['PC2'], mode='markers',
                                 marker=dict(size=10, color=agrupamento.labels_ + 1, opacity=0.8,
                                             colorscale='Viridis'))])

# Configurar layout
fig.update_layout(title='Clustering de intentos',
                  xaxis_title='PC1',
                  yaxis_title='PC2',
                  width=800, height=600)

# Mostrar gráfica
fig.show()"""


"from sklearn.cluster import DBSCAN\nfrom sklearn.preprocessing import StandardScaler\n\n#Coger las columnas necesarias para crear la matriz de características\nmatriz_caracteristicas = df_pca[['PC1', 'PC2']].values\n\n# Normalizar los datos\n#matriz_caracteristicas_norm = StandardScaler().fit_transform(matriz_caracteristicas)\n\n# Definir el modelo\nmodel = DBSCAN(eps=0.7, min_samples=5)\n\n# Ajustar el modelo a los datos\nagrupamento = model.fit(matriz_caracteristicas)\n\n# Crear la figura\nfig = go.Figure(data=[go.Scatter(x=df_pca['PC1'], y=df_pca['PC2'], mode='markers',\n                                 marker=dict(size=10, color=agrupamento.labels_ + 1, opacity=0.8,\n                                             colorscale='Viridis'))])\n\n# Configurar layout\nfig.update_layout(title='Clustering de intentos',\n                  xaxis_title='PC1',\n                  yaxis_title='PC2',\n                  width=800, height=600)\n\n# Mostrar gráfica\nfig.show()"

# Con Hierarchical Agglomerative Clustering (HAC):

In [24]:
"""import numpy as np
import plotly.graph_objects as go
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

#Tecnicas de principal component analysis
#conjunto minimo de datos que explican una cirscuntancia
#Claro, el Principal Component Analysis (PCA) es una técnica de reducción de dimensionalidad que se utiliza en el análisis de datos y en el aprendizaje automático. El objetivo principal de PCA es transformar un conjunto de variables correlacionadas en un conjunto de variables no correlacionadas llamadas componentes principales.

#PCA es útil cuando se trabaja con un conjunto de datos con muchas variables, y se desea reducir la cantidad de variables manteniendo la mayor cantidad de información posible. Esto se logra al encontrar las direcciones en las que los datos tienen la mayor varianza y proyectar los datos en estas direcciones.

#El proceso de PCA se puede resumir en los siguientes pasos:

#Estandarizar los datos: Esto implica escalar las variables para que tengan una media de 0 y una desviación estándar de 1.
#Calcular la matriz de covarianza: La matriz de covarianza es una matriz cuadrada que contiene las covarianzas entre cada par de variables.
#Calcular los eigenvectores y eigenvalores de la matriz de covarianza: Los eigenvectores representan las direcciones en las que los datos tienen la mayor varianza, y los eigenvalores representan la magnitud de la varianza en cada dirección.
#Ordenar los eigenvectores según sus eigenvalores correspondientes en orden descendente.
#Seleccionar los primeros k eigenvectores, donde k es el número de componentes principales que se desea mantener.
#Proyectar los datos en el espacio de los k eigenvectores seleccionados.
#El resultado es un conjunto de datos transformado con menos dimensiones, que conserva la mayor cantidad de información posible del conjunto de datos original.


#elegir 2 sistemas cada uno, 1 mas, resumen. tracksrls frances si. uno de codigo abierto
#ver funcionalidades y visualizacion y analisis

#predecir si al jugador le va a ir bien o no segun los datos 

#Coger las columnas necesarias para crear la matriz de características
matriz_caracteristicas = df_pca[['PC1', 'PC2']].values

# Normalizar los datos
#matriz_caracteristicas_norm = StandardScaler().fit_transform(matriz_caracteristicas)

# Calcular la matriz de enlaces
Z = linkage(matriz_caracteristicas, method='ward')

# Calcular los valores de silueta para diferentes umbrales
scores = []
for i in range(2, len(Z)):
    clusters = fcluster(Z, t=i, criterion='maxclust')
    score = silhouette_score(matriz_caracteristicas, clusters)
    scores.append(score)

# Encontrar el umbral que maximiza el valor de silueta
umbral = np.argmax(scores) + 2

# Asignar los clusters en base al umbral
agrupamiento = fcluster(Z, t=umbral, criterion='maxclust')

# Crear la figura
fig = go.Figure(data=[go.Scatter(x=df_pca['PC1'], y=df_pca['PC2'], mode='markers',
                                 marker=dict(size=10, color=agrupamento.labels_ + 1, opacity=0.8,
                                             colorscale='Viridis'))])

# Configurar layout
fig.update_layout(title='Clustering de intentos',
                  xaxis_title='PC1',
                  yaxis_title='PC2',
                  width=800, height=600)

# Mostrar gráfica
fig.show()"""


"import numpy as np\nimport plotly.graph_objects as go\nfrom scipy.cluster.hierarchy import linkage, fcluster\nfrom sklearn.metrics import silhouette_score\nfrom sklearn.preprocessing import StandardScaler\n\n#Tecnicas de principal component analysis\n#conjunto minimo de datos que explican una cirscuntancia\n#Claro, el Principal Component Analysis (PCA) es una técnica de reducción de dimensionalidad que se utiliza en el análisis de datos y en el aprendizaje automático. El objetivo principal de PCA es transformar un conjunto de variables correlacionadas en un conjunto de variables no correlacionadas llamadas componentes principales.\n\n#PCA es útil cuando se trabaja con un conjunto de datos con muchas variables, y se desea reducir la cantidad de variables manteniendo la mayor cantidad de información posible. Esto se logra al encontrar las direcciones en las que los datos tienen la mayor varianza y proyectar los datos en estas direcciones.\n\n#El proceso de PCA se puede resumir en los si

# Con Spectral Clustering

In [25]:
"""from sklearn.cluster import SpectralClustering

# Eliminar columnas innecesarias antes de crear la matriz de características
matriz_caracteristicas = df_pca[['PC1', 'PC2']].values

# Normalizar los datos
matriz_caracteristicas_norm = StandardScaler().fit_transform(matriz_caracteristicas)

# Definimos el modelo
model = SpectralClustering(n_components=None, random_state=None)

# Axustamos el modelo aos datos
agrupamiento = model.fit_predict(matriz_caracteristicas_norm)

# Crear la figura
fig = go.Figure(data=[go.Scatter(x=df_pca['PC1'], y=df_pca['PC2'], mode='markers',
                                 marker=dict(size=10, color=agrupamento.labels_ + 1, opacity=0.8,
                                             colorscale='Viridis'))])

# Configurar layout
fig.update_layout(title='Clustering de intentos',
                  xaxis_title='PC1',
                  yaxis_title='PC2',
                  width=800, height=600)

# Mostrar gráfica
fig.show()"""


"from sklearn.cluster import SpectralClustering\n\n# Eliminar columnas innecesarias antes de crear la matriz de características\nmatriz_caracteristicas = df_pca[['PC1', 'PC2']].values\n\n# Normalizar los datos\nmatriz_caracteristicas_norm = StandardScaler().fit_transform(matriz_caracteristicas)\n\n# Definimos el modelo\nmodel = SpectralClustering(n_components=None, random_state=None)\n\n# Axustamos el modelo aos datos\nagrupamiento = model.fit_predict(matriz_caracteristicas_norm)\n\n# Crear la figura\nfig = go.Figure(data=[go.Scatter(x=df_pca['PC1'], y=df_pca['PC2'], mode='markers',\n                                 marker=dict(size=10, color=agrupamento.labels_ + 1, opacity=0.8,\n                                             colorscale='Viridis'))])\n\n# Configurar layout\nfig.update_layout(title='Clustering de intentos',\n                  xaxis_title='PC1',\n                  yaxis_title='PC2',\n                  width=800, height=600)\n\n# Mostrar gráfica\nfig.show()"

In [26]:
"""df['cluster'] = agrupamento.labels_
df"""

"df['cluster'] = agrupamento.labels_\ndf"

# Anteriores

In [27]:
"""

# Pivotear el DataFrame para tener un formato largo
intentos_df_long = pd.melt(intentosJugadores.reset_index(), id_vars=['index'], var_name='Jugador', value_name='Intentos')
intentos_df_long.columns = ['Nivel', 'Jugador', 'Intentos']

# Agrupar los intentos de cada jugador por nivel y tomar la media
intentos_jugador_nivel = intentos_df_long.groupby(['Jugador', 'Nivel'])['Intentos'].mean().unstack()

# Unir la tabla de intentos de cada jugador con los datos originales
df = data_filtrado.merge(intentos_jugador_nivel, left_on='token', right_index=True)

# Seleccionar las variables a utilizar para el clustering
X = df.iloc[:, 3:] # Seleccionar las columnas correspondientes a los niveles

# Imputar los valores faltantes
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

# Normalizar las variables utilizando StandardScaler
scaler = StandardScaler()
X_norm = scaler.fit_transform(X_imputed)

# Definir el número de clusters y entrenar el modelo de K-means
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(X_norm)

# Asignar a cada jugador el cluster correspondiente
clusters = kmeans.labels_
df['Cluster'] = clusters

# Calcular la media de los números de intentos en cada cluster
means = df.groupby(['G03Q02', 'G03Q01', 'Cluster'])[X.columns].mean()

# Mostrar los resultados
print(means) #Dar a abrir en un editor de texto para ver mejor la tabla
#intentosJugadores
"""

"\n\n# Pivotear el DataFrame para tener un formato largo\nintentos_df_long = pd.melt(intentosJugadores.reset_index(), id_vars=['index'], var_name='Jugador', value_name='Intentos')\nintentos_df_long.columns = ['Nivel', 'Jugador', 'Intentos']\n\n# Agrupar los intentos de cada jugador por nivel y tomar la media\nintentos_jugador_nivel = intentos_df_long.groupby(['Jugador', 'Nivel'])['Intentos'].mean().unstack()\n\n# Unir la tabla de intentos de cada jugador con los datos originales\ndf = data_filtrado.merge(intentos_jugador_nivel, left_on='token', right_index=True)\n\n# Seleccionar las variables a utilizar para el clustering\nX = df.iloc[:, 3:] # Seleccionar las columnas correspondientes a los niveles\n\n# Imputar los valores faltantes\nimputer = SimpleImputer(strategy='median')\nX_imputed = imputer.fit_transform(X)\n\n# Normalizar las variables utilizando StandardScaler\nscaler = StandardScaler()\nX_norm = scaler.fit_transform(X_imputed)\n\n# Definir el número de clusters y entrenar el m

In [28]:
"""tiemposJugadores = pd.DataFrame(tiemposMedios["tiemposIndividuales"])

# Aplicar la función int() solamente a los valores que no son NaN en el DataFrame
tiemposJugadores = tiemposJugadores.applymap(lambda x: int(x) if x == x else x)

# Pivotear el DataFrame para tener un formato largo
tiempos_df_long = pd.melt(tiemposJugadores.reset_index(), id_vars=['index'], var_name='Jugador', value_name='Intentos')
tiempos_df_long.columns = ['Nivel', 'Jugador', 'Intentos']

# Agrupar los intentos de cada jugador por nivel y tomar la media
tiempos_jugador_nivel = tiempos_df_long.groupby(['Jugador', 'Nivel'])['Intentos'].mean().unstack()

# Unir la tabla de intentos de cada jugador con los datos originales
df = data_filtrado.merge(tiempos_jugador_nivel, left_on='token', right_index=True)

# Seleccionar las variables a utilizar para el clustering
X = df.iloc[:, 3:] # Seleccionar las columnas correspondientes a los niveles

# Imputar los valores faltantes
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

# Normalizar las variables utilizando StandardScaler
scaler = StandardScaler()
X_norm = scaler.fit_transform(X_imputed)

# Definir el número de clusters y entrenar el modelo de K-means
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(X_norm)

# Asignar a cada jugador el cluster correspondiente
clusters = kmeans.labels_
df['Cluster'] = clusters

# Calcular la media de los números de intentos en cada cluster
means = df.groupby(['G03Q02', 'G03Q01', 'Cluster'])[X.columns].mean()

# Mostrar los resultados
print(means)
# tiemposJugadores"""


'tiemposJugadores = pd.DataFrame(tiemposMedios["tiemposIndividuales"])\n\n# Aplicar la función int() solamente a los valores que no son NaN en el DataFrame\ntiemposJugadores = tiemposJugadores.applymap(lambda x: int(x) if x == x else x)\n\n# Pivotear el DataFrame para tener un formato largo\ntiempos_df_long = pd.melt(tiemposJugadores.reset_index(), id_vars=[\'index\'], var_name=\'Jugador\', value_name=\'Intentos\')\ntiempos_df_long.columns = [\'Nivel\', \'Jugador\', \'Intentos\']\n\n# Agrupar los intentos de cada jugador por nivel y tomar la media\ntiempos_jugador_nivel = tiempos_df_long.groupby([\'Jugador\', \'Nivel\'])[\'Intentos\'].mean().unstack()\n\n# Unir la tabla de intentos de cada jugador con los datos originales\ndf = data_filtrado.merge(tiempos_jugador_nivel, left_on=\'token\', right_index=True)\n\n# Seleccionar las variables a utilizar para el clustering\nX = df.iloc[:, 3:] # Seleccionar las columnas correspondientes a los niveles\n\n# Imputar los valores faltantes\nimputer

In [29]:
"""media_intentos = df.groupby(['G03Q02', 'G03Q01', 'Cluster'])[X.columns].mean().reset_index()
media_intentos = pd.melt(media_intentos, id_vars=['G03Q02', 'G03Q01', 'Cluster'], var_name='Nivel', value_name='Intentos')

import seaborn as sns
import matplotlib.pyplot as plt

# Crear una figura para cada nivel
for nivel in intentos_df_long['Nivel'].unique():
    # Seleccionar los datos del nivel y generar una tabla pivote con los valores medios de intentos por género
    df_nivel = media_intentos[media_intentos['Nivel'] == nivel]
    pivot_table = pd.pivot_table(df_nivel, values='Intentos', index=['G03Q01'], columns=['Cluster'], aggfunc=np.mean)
    
    # Crear un gráfico de barras apiladas para comparar los valores medios de intentos por género en cada cluster
    sns.catplot(x='G03Q01', y='Intentos', hue='Cluster', data=df_nivel, kind='bar', ci=None, legend_out=False)
    plt.title(f'Nivel {nivel}')
    plt.show()"""


"media_intentos = df.groupby(['G03Q02', 'G03Q01', 'Cluster'])[X.columns].mean().reset_index()\nmedia_intentos = pd.melt(media_intentos, id_vars=['G03Q02', 'G03Q01', 'Cluster'], var_name='Nivel', value_name='Intentos')\n\nimport seaborn as sns\nimport matplotlib.pyplot as plt\n\n# Crear una figura para cada nivel\nfor nivel in intentos_df_long['Nivel'].unique():\n    # Seleccionar los datos del nivel y generar una tabla pivote con los valores medios de intentos por género\n    df_nivel = media_intentos[media_intentos['Nivel'] == nivel]\n    pivot_table = pd.pivot_table(df_nivel, values='Intentos', index=['G03Q01'], columns=['Cluster'], aggfunc=np.mean)\n    \n    # Crear un gráfico de barras apiladas para comparar los valores medios de intentos por género en cada cluster\n    sns.catplot(x='G03Q01', y='Intentos', hue='Cluster', data=df_nivel, kind='bar', ci=None, legend_out=False)\n    plt.title(f'Nivel {nivel}')\n    plt.show()"